## Setup

Load libraries

In [1]:
import glob as gb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Loading the Digits Dataset

The digits dataset has six views and each view has 2000 observations.

In [2]:
# load the data
all_views = gb.glob('./data/multiple+features/mfeat-*')

## working with two views
mapper = map(lambda filename: pd.read_csv(filename, index_col=None, sep='\s+', header=None), all_views)

# Multiview
X_mv=list(mapper)

## Singleview
X_sv=pd.concat(X_mv, axis=1, ignore_index=True)

## Output
y = np.repeat(range(0,10), 200)

In [9]:
print("Number of features in each view")

for idx, vinw in enumerate(X_mv):
    print(f"View {idx+1}: {vinw.shape[1]} features")

Number of features in each view
View 1: 6 features
View 2: 216 features
View 3: 76 features
View 4: 47 features
View 5: 64 features
View 6: 240 features


In [6]:
X_mv[0].shape

(2000, 6)